# ANÁLISE PREDITIVA DO PREÇO DE VEÍCULOS 

## Descrição:
Segunda atividade do módulo 4 da disciplina 13 - Introdução a Ciência de Dados e Inteligência Artificial, do curso Talento Tech - PR.

## Objetivo:
Implementar um modelo para predição da condição do fechamento do Bitcoin, em relação ao dia anterior.

## Ferramentas utilizadas:
O projeto foi desenvolvido em python, com as seguintes bibliotecas:
* Yfinance, para coleta de preços
* Pandas, para manipulação dos dados
* Scikit-Learn, para utilizar os modelos de previsão

---

### Metodologia

A análise foi feita em 4 períodos de tempo diferentes. Além dos dados de abertura, fechamento, máxima e mínima, também foram utilizadas as Médias Móveis Simples e Ponderadas. A Média Simples funciona para dar uma visão geral sobre a tendência de curto prazo, enquanto a Média Ponderada considera uma visão de prazo maior.

---

### Preparo dos dados

In [38]:
import pandas as pd
import numpy as np

data_6m = pd.read_csv("data/data_6_months.csv")
data_3m = pd.read_csv("data/data_3_months.csv")
data_1y = pd.read_csv("data/data_1_year.csv")
data_2y = pd.read_csv("data/data_2_years.csv")

def get_dataframe(df: pd.DataFrame, MMS_PARAM, WMA_PARAM):
    MAX_RANGE = max(MMS_PARAM, WMA_PARAM) - 1

    for i in range(MAX_RANGE, len(df)):
        df.loc[i - (MMS_PARAM - 1): i]["Close"]
        
        mms = df.loc[i - (MMS_PARAM - 1): i]["Close"].mean()
        wma = np.average(df.loc[i - (WMA_PARAM - 1): i]["Close"], weights=[k + 1 for k in range(WMA_PARAM)])

        df.at[i, "MMS"] = mms
        df.at[i, "WMA"] = wma
        
        df.at[i, "High"] = 1 if df.iloc[i - 1]["Close"] < df.iloc[i]["Close"] else 0
        
    df = df.loc[MAX_RANGE:].reset_index(drop=True)
    df["High"] = df["High"].astype(int)
    return df


---

### Treinamento e acurácia

#### Random Forest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_best_params_rf(data: pd.DataFrame):
    MMS_PARAMS = [4,5,6,7,8,9,10]
    WMA_PARAMS = [4,5,6,7,8,9,10]
    results = []
    
    for MMS_PARAM in MMS_PARAMS:
        for WMA_PARAM in WMA_PARAMS:
            df = get_dataframe(data, MMS_PARAM, WMA_PARAM)
            
            y = df["High"]
            x = df[["Close", "Low", "Open", "Volume", "MMS", "WMA"]]
            
            scaler = StandardScaler()
            x = scaler.fit_transform(x)

            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
            
            
            model = RandomForestClassifier(random_state=42)
            model.fit(x_train, y_train)
            
            preds = model.predict(x_test)
            
            score = accuracy_score(y_test, preds)
            
            results.append({"score": score, "MMS_PARAM": MMS_PARAM, "WMA_PARAM": WMA_PARAM})
            
    return max(results, key=lambda x: x["score"])

results_6m_rf = get_best_params_rf(data_6m)
results_3m_rf = get_best_params_rf(data_3m)
results_1y_rf = get_best_params_rf(data_1y)
results_2y_rf = get_best_params_rf(data_2y)
            


In [5]:
print("3 meses:", results_3m_rf)
print("6 meses:", results_6m_rf)
print("1 ano:", results_1y_rf)
print("2 anos:", results_2y_rf)

3 meses: {'score': 0.6363636363636364, 'MMS_PARAM': 5, 'WMA_PARAM': 8}
6 meses: {'score': 0.84, 'MMS_PARAM': 4, 'WMA_PARAM': 5}
1 ano: {'score': 0.8, 'MMS_PARAM': 4, 'WMA_PARAM': 6}
2 anos: {'score': 0.5555555555555556, 'MMS_PARAM': 9, 'WMA_PARAM': 5}


#### Regressão Logística

In [6]:
from sklearn.linear_model import LogisticRegression

def get_best_params_lg(data: pd.DataFrame):
    MMS_PARAMS = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    WMA_PARAMS = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    results = []
    
    for MMS_PARAM in MMS_PARAMS:
        for WMA_PARAM in WMA_PARAMS:
            df = get_dataframe(data, MMS_PARAM, WMA_PARAM)
            
            y = df["High"]
            x = df[["Close", "Low", "Open", "Volume", "MMS", "WMA"]]
            
            scaler = StandardScaler()
            x = scaler.fit_transform(x)

            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
            
            
            model = LogisticRegression(random_state=42)
            model.fit(x_train, y_train)
            
            preds = model.predict(x_test)
            
            score = accuracy_score(y_test, preds)
            
            results.append({"score": score, "MMS_PARAM": MMS_PARAM, "WMA_PARAM": WMA_PARAM})
            
    return max(results, key=lambda x: x["score"])

results_6m_lg = get_best_params_lg(data_6m)
results_3m_lg = get_best_params_lg(data_3m)
results_1y_lg = get_best_params_lg(data_1y)
results_2y_lg = get_best_params_lg(data_2y)
            


In [7]:
print("3 meses:", results_3m_lg)
print("6 meses:", results_6m_lg)
print("1 ano:", results_1y_lg)
print("2 anos:", results_2y_lg)

3 meses: {'score': 0.8181818181818182, 'MMS_PARAM': 6, 'WMA_PARAM': 12}
6 meses: {'score': 0.8260869565217391, 'MMS_PARAM': 3, 'WMA_PARAM': 13}
1 ano: {'score': 0.851063829787234, 'MMS_PARAM': 20, 'WMA_PARAM': 3}
2 anos: {'score': 0.7755102040816326, 'MMS_PARAM': 10, 'WMA_PARAM': 18}


---

## Conclusão

Embora os dados de teste estejam presentes em um cenário de maior volatilidade do Bitcoin, os modelos se comportaram de forma satisfatória.

### Random Forest
O Random Forest se adequou melhor aos períodos de média duração, adotando Médias Móveis Simples e Ponderadas curtas, sinalizando um possível foco em movimentos curtos. Isso pode ser observado no maior intervalo de tempo, onde a acurácia do modelo é menor.

### Regressão Logística
A regressão logística obteve bons resultados em todos os períodos de tempo analisados. No geral, o modelo funciona melhor com Médias Móveis Simples, capturando movimentos de curto prazo. Além disso, ele se adequou melhor com Médias Móveis Ponderadas mais longas, suavizando os dados e capturando também as tendências de longo prazo. 